In [10]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
import spacy
nlp = spacy.load("en_core_web_sm")
import pycountry
import re
import json
from bert_score import score as bert_score


/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Load Model files

In [11]:
df = pd.read_pickle('../models/df_embed_EN.pkl')

### Load Enviroment files

In [12]:
# Load environment variables
load_dotenv()

True

In [13]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3>globals</h3>

In [14]:
# test_query="What are the sustainable energy priorities for UNDP?"
test_query = 'What is the Human Development Index (HDI) value for Albania as mentioned in the document?'


<h3> helper functions </h3>

In [15]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [16]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
 
    """
    entity_list = callOpenAI(prompt)   
    return entity_list

# Test usage
# test_query = "What are the sustainable energy for UNDP?"
# entity_list = extractEntitiesFromQuery(test_query)
# print(entity_list)

In [17]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "What is the role of Paris Agreement in sustainable energy development?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

{'relations': 'The Paris Agreement is related to sustainable energy development.', 'entities': {'Paris Agreement': 'The Paris Agreement is an international treaty adopted in 2015 by nearly every country in the world. Its goal is to combat climate change by limiting global warming to well below 2 degrees Celsius above pre-industrial levels, and to pursue efforts to limit the temperature increase to 1.5 degrees Celsius.', 'sustainable energy development': 'Sustainable energy development refers to the process of harnessing and utilizing renewable energy sources in a manner that minimizes environmental impact and promotes long-term viability. It involves the adoption of clean technologies, such as solar, wind, and hydro power, to meet energy needs while reducing greenhouse gas emissions and preserving natural resources.'}}


In [18]:
# def find_mentioned_countries(text):
#     doc = nlp(text)
#     countries = set()
    
#     for ent in doc.ents:
#         if ent.label_ == "GPE":  # GPE stands for "Geopolitical Entity"
#             countries.add(ent.text)
    
#     return list(countries)

 
def find_mentioned_countries(text):
    countries = set()
    
    # Tokenize the text using regular expressions to preserve punctuation marks
    words = re.findall(r'\w+|[^\w\s]', text)
    text = ' '.join(words)  # Join the tokens back into a string
    
    for word in text.split():
        try:
            country = pycountry.countries.get(name=word) #pycountry.countries.lookup(word)
            if country != None : 
               countries.add(country.name)
        except LookupError:
            pass
    
    return list(countries)

# Example 
# user_query = 'Give me a summary of the goals UNDP wants to achieve in 10 years and the energy plans for Philippines and Angola'
# mentioned_countries = find_mentioned_countries(user_query)
# mentioned_countries

In [19]:
def filter_country(user_query):
    mentioned_countries = find_mentioned_countries(user_query)
    # print(mentioned_countries)
    # Check if mentioned_countries is not empty
    if mentioned_countries:
        country = mentioned_countries[0]
        return df[df['Country Name'] == country]
    else:
        # Handle the case where no countries were mentioned
        return None  # Or return an empty DataFrame or any other suitable value


# Example 
# test_query="What are the main goals of Niger's Vision 2035?"
# filtered_country = filter_country(test_query)
# filtered_country

In [20]:
def search_embeddings(user_query):
    df_filtered = filter_country(user_query) if filter_country(user_query) is not None else None
    
    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)

        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None


In [21]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [22]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0
    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 10
        if count == 20:
            break

    return result_dict


In [23]:
## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    # if qs != None :
    if qs[0] is not None:
        result_structure = map_to_structure(qs)
        return result_structure
    else : 
        return []
#test usage
excerpts_dict=semanticSearchModule(test_query)
# print(f"""excerpts_dict {excerpts_dict}""")

#Return top 10-20 most related 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/excerpts.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

print(f"Excerpts saved to {json_filename}")

Excerpts saved to outputs/excerpts.json


In [24]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

#test usage
indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}


In [25]:
## module to generate query ideas

def queryIdeationModule(user_query): # lower priority
    
    # Generate query ideas using OpenAI GPT-3
    prompt = f"""Generate query ideas based on the user query: {user_query}
    -You Must return output in array format e.g ['idea 1', 'idea2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    response = callOpenAI(prompt)
    return response

#test usage
# query_idea_list=queryIdeationModule(test_query)
# print(query_idea_list)

<h3> synthesis module </h3>

    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 


In [26]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, Sources, and entity and relation info to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
    Entity and Relation info: {entities_dict}

    - Answer output must be properly formatted using HTML. 
    - Don't include <html>, <script>, <link> or <body> tags. Only text formating tags should be allowed. e.g h1..h3, p, anchor, etc.
    - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
    - The citations anchor should be near the excerpt not following each other.
    - Use the anchor tag for the citation links and should link to the document link. for example Undp operates in afganistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
    - The text in the anchor tag should be citation number not document title.
    - You can reference multitple citations based sources
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [27]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
    
    ##run processing modules (in parallel)
    entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "query_ideas":query_idea_list,
        "entities":list(entities_dict["entities"].keys())       
    }
    
    return response

# test usage
test_query = "What percentage of Niger's population has access to electricity for productive use"
response=handleApiCall(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/synthesis_output.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

Synthesis saved to outputs/synthesis_output.json


<h3>testing</h3>

In [28]:
## next step, develop automated testing for all modules
## iterate through test_queries and build automated tests to score results

# open testing dataset with queries and expected results
# test_queries_df=pd.read_csv("../testing/energy_ai_test_dataset_v0.csv")



In [29]:
# !pip install bert_score

In [30]:

# # Refactor: https://openreview.net/pdf?id=SkeHuCVFDr
# def calculate_scores(data):
#     for entry in data:
#         query = entry['query']
#         sample_answer = entry['sample_answer']
#         moonshot_model_answer = handleApiCall(query) 
#         P, F, R = bert_score([sample_answer], [moonshot_model_answer['answer']], lang='en', verbose=True)
#         entry['moonshot_model_answer'] = moonshot_model_answer['answer']
#         meteor_score_value = 0 #meteor_score([sample_answer], query)
#         entry['bert_score'] = F
#         # entry['meteor_score'] = meteor_score_value

# # Load data from JSON file
# with open('../testing/queries.json', 'r') as file:
#     data = json.load(file)

# # Calculate scores
# calculate_scores(data)

# # Convert tensor scores to lists
# for entry in data:
#     entry['bert_score'] = entry['bert_score'].tolist()

# # Print updated data with scores
# # print(json.dumps(data, indent=4))
# # Save updated data to a JSON file
# with open('../testing/test_output.json', 'w') as file:
#     json.dump(data, file, indent=4)


In [31]:
import csv
import json

# Refactor: https://openreview.net/pdf?id=SkeHuCVFDr
def calculate_scores(data):
    output_data = []
    for entry in data:
        query = entry['query']
        sample_answer = entry['sample_answer']
        moonshot_model_answer = {}  # Initialize as empty dictionary
        if query:  # Check if query is non-empty before calling handleApiCall
            moonshot_model_answer = handleApiCall(query)  # Assuming handleApiCall is defined elsewhere
        P, F, R = bert_score([sample_answer], [moonshot_model_answer.get('answer', '')], lang='en', verbose=True)
        entry['moonshot_model_answer'] = moonshot_model_answer.get('answer', '')  # Use get method to handle missing key
        meteor_score_value = 0  # meteor_score([sample_answer], query)
        entry['bert_score'] = F
        # entry['meteor_score'] = meteor_score_value
        output_data.append(entry)
    return output_data

# Load data from CSV file
def load_data_from_csv(csv_file):
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        return list(reader)

# Specify CSV file paths
input_csv_file = "../testing/queries.csv"
output_json_file = "../testing/test_output.json"

# Load data from CSV file
data = load_data_from_csv(input_csv_file)

# Calculate scores
data_with_scores = calculate_scores(data)

# Write data with scores to JSON file
with open(output_json_file, mode='w') as file:
    json.dump(data_with_scores, file, indent=4)

print("JSON file with scores has been created successfully.")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.10s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 40.65it/s]


...

100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 212.09it/s]


done in 0.90 seconds, 1.12 sentences/sec


TypeError: Object of type Tensor is not JSON serializable

# todo
- Add GPT  answer option 
- Prompts testings  - use prompt engineering frameworks and compare performance 
- Move test queries to CSV format * @Ben


